In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import os
os.environ['KERAS_BACKEND']='tensorflow' # set backend

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras.datasets import imdb
from keras import backend as K

Using TensorFlow backend.


In [3]:
# set parameters:
max_features = 5000
maxlen = 100

In [4]:
# look into preprocessor.py file for details
from preprocessor import Index_Sequence, accuracy, indicator_to_matrix

In [5]:
yelp_data = Index_Sequence("yelp_academic_dataset_review.json", "text", "stars", 500,maxlen)

In [6]:
embedding_dims = 50
nb_filter = 250
filter_length = 3
hidden_dims = 250

nb_classes = len(yelp_data.docs_label_index)

In [7]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.2))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use max pooling:
model.add(MaxPooling1D(pool_length=model.output_shape[1]))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

Build model...


In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
embedding_1 (Embedding)            (None, 100, 50)     250000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)    (None, 98, 250)     37750       embedding_1[0][0]                
____________________________________________________________________________________________________
maxpooling1d_1 (MaxPooling1D)      (None, 1, 250)      0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
flatten_1 (Flatten)                (None, 250)         0           maxpooling1d_1[0][0]             
___________________________________________________________________________________________

In [10]:
#from preprocessor import Dataset, pad_vec_sequences, pad_sent_sequences
from sklearn import preprocessing
from sklearn import cross_validation

In [11]:
# data split
x_train, x_test, y_train, y_test = cross_validation.train_test_split(yelp_data.X_doc_seq,yelp_data.Y_doc_seq,test_size=0.2)

# create appropirate matrix (hot encoded) response
y_train, y_test = [indicator_to_matrix(x,yelp_data.docs_label_index)  for x in (y_train, y_test)]

In [12]:
# model specs
batch_size = 32
nb_epoch = 10

# train model
model.fit(x_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(x_test, y_test))

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients.py:89: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 400 samples, validate on 100 samples
Epoch 1/10
400/400 [==============================] - 10s - loss: 1.5873 - acc: 0.2675 - val_loss: 1.5599 - val_acc: 0.3700
Epoch 2/10
400/400 [==============================] - 9s - loss: 1.5664 - acc: 0.2725 - val_loss: 1.5595 - val_acc: 0.3700
Epoch 3/10
400/400 [==============================] - 9s - loss: 1.5529 - acc: 0.2725 - val_loss: 1.5583 - val_acc: 0.3700
Epoch 4/10
400/400 [==============================] - 10s - loss: 1.5345 - acc: 0.3525 - val_loss: 1.5513 - val_acc: 0.3600
Epoch 5/10
400/400 [==============================] - 9s - loss: 1.4941 - acc: 0.3400 - val_loss: 1.5185 - val_acc: 0.3500
Epoch 6/10
400/400 [==============================] - 9s - loss: 1.4159 - acc: 0.3850 - val_loss: 1.5096 - val_acc: 0.3000
Epoch 7/10
400/400 [==============================] - 9s - loss: 1.3409 - acc: 0.4800 - val_loss: 1.4915 - val_acc: 0.3300
Epoch 8/10
400/400 [==============================] - 9s - loss: 1.2274 - acc: 0.5425 - val

/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormat

In [13]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

100/100 [==============================] - 1s     
Test loss: 1.43675780296
Test accuracy: 0.34


In [ ]:
model.save('yelp_cnn_hidden_epocs.h5') # can't save this why?

In [ ]:
from keras.models import load_model
current_model = load_model('yelp_cnn_hidden_epocs.h5')